In [1]:
import os
import sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

import pandas as pd
import numpy as np
import datetime
import torch
import math

from Clust.clust.ML.regression_JS import ada_rnn
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.ML.common.common import p3_training as p3
from Clust.clust.transformation.purpose.machineLearning import LSTMData

from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.data import df_set_data
from Clust.clust.data import data_interface

from Clust.setting import influx_setting_KETI as ins
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)

from clust.preprocessing.dataPreprocessing import DataPreprocessing
from Clust.clust.preprocessing.imputation.basicMethod import BasicImputation
from Clust.clust.transformation.purpose.machineLearning import LSTMData

from Clust.clust.ML.tool import scaler as ml_scaler

# Set Param

In [2]:
feature_list = ["out_blacksphere_temp", "out_coai", "out_h2s", 'out_humi', 'out_nh3', 'out_pm10', 'out_temp', 'out_ultraviolet_rays']
target_col = "out_pm10"

param = {
    "x_features" : 8, 
    "class_num" : 1, 
    "dropout" : 0.0, 
    "len_seq" : 24, 
    "loss_type" : "adv",
    "hidden_size" : 64,
    "num_layers" : 2
}

shuffle = True
model_file_path = "./ada_rnn_model/test_adarnn_model_by_origin_data"

inference_start_time = "2022-04-30 00:00:00"
inference_end_time = "2022-05-01 00:00:00"

bk_name = "air_outdoor_kweather"
ms_name = "OC3CL200010"

# get Data

## Data Read

In [3]:
origin_data = db_client.get_data_by_time(inference_start_time, inference_end_time, bk_name, ms_name)

In [4]:
origin_data

,out_blacksphere_temp,out_co,out_coai,out_coci,out_h2s,out_humi,out_light_intensity,out_nh3,out_no2,out_noise,out_o3,out_pm10,out_pm25,out_so2,out_temp,out_ultraviolet_rays
time,,,,,,,,,,,,,,,,
2022-04-30 00:00:00+00:00,10.3,-9999.0,100.0,100.0,0.0,75.0,-9999.0,0.0,-9999.0,42.0,-9999.0,0.0,0.0,-9999.0,11.2,1.3
2022-04-30 00:02:00+00:00,10.4,-9999.0,100.0,100.0,0.0,75.0,-9999.0,0.0,-9999.0,41.0,-9999.0,0.0,0.0,-9999.0,11.3,1.3
2022-04-30 00:04:00+00:00,10.3,-9999.0,100.0,100.0,0.0,74.0,-9999.0,0.0,-9999.0,41.0,-9999.0,0.0,0.0,-9999.0,11.3,1.3
2022-04-30 00:06:00+00:00,10.3,-9999.0,100.0,100.0,0.0,74.0,-9999.0,0.0,-9999.0,41.0,-9999.0,0.0,0.0,-9999.0,11.3,1.3
2022-04-30 00:08:00+00:00,10.6,-9999.0,100.0,100.0,0.0,77.0,-9999.0,0.0,-9999.0,41.0,-9999.0,0.0,0.0,-9999.0,11.4,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-30 23:50:00+00:00,9.1,-9999.0,79.0,76.0,0.0,100.0,-9999.0,0.0,-9999.0,35.0,-9999.0,60.0,41.0,-9999.0,8.4,1.3
2022-04-30 23:52:00+00:00,9.1,-9999.0,79.0,76.0,0.0,100.0,-9999.0,0.0,-9999.0,34.0,-9999.0,59.0,41.0,-9999.0,8.4,1.3
2022-04-30 23:54:00+00:00,9.1,-9999.0,79.0,76.0,0.0,100.0,-9999.0,0.0,-9999.0,35.0,-9999.0,60.0,41.0,-9999.0,8.4,1.3


In [5]:
origin_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 714 entries, 2022-04-30 00:00:00+00:00 to 2022-04-30 23:58:00+00:00
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   out_blacksphere_temp  714 non-null    float64
 1   out_co                714 non-null    float64
 2   out_coai              714 non-null    float64
 3   out_coci              714 non-null    float64
 4   out_h2s               714 non-null    float64
 5   out_humi              714 non-null    float64
 6   out_light_intensity   714 non-null    float64
 7   out_nh3               714 non-null    float64
 8   out_no2               714 non-null    float64
 9   out_noise             714 non-null    float64
 10  out_o3                714 non-null    float64
 11  out_pm10              714 non-null    float64
 12  out_pm25              714 non-null    float64
 13  out_so2               714 non-null    float64
 14  out_temp              714

In [6]:
# Check Frequency
freq = db_client.get_freq(bk_name, ms_name)
print(freq)

0 days 00:02:00


In [7]:
origin_data = origin_data[feature_list]

## Data Refine

In [8]:
# fill in the missing time index
refine_param = {'removeDuplication': {'flag': True} , 'staticFrequency': {'flag': True, 'frequency': "1H"}}
data_refine = DataPreprocessing().get_refinedData(origin_data, refine_param)

In [9]:
data_refine.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2022-04-30 00:00:00+00:00 to 2022-04-30 23:00:00+00:00
Freq: H
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   out_blacksphere_temp  24 non-null     float64
 1   out_coai              24 non-null     float64
 2   out_h2s               24 non-null     float64
 3   out_humi              24 non-null     float64
 4   out_nh3               24 non-null     float64
 5   out_pm10              24 non-null     float64
 6   out_temp              24 non-null     float64
 7   out_ultraviolet_rays  24 non-null     float64
dtypes: float64(8)
memory usage: 1.7 KB


In [10]:
data_refine.head()

,out_blacksphere_temp,out_coai,out_h2s,out_humi,out_nh3,out_pm10,out_temp,out_ultraviolet_rays
time,,,,,,,,
2022-04-30 00:00:00+00:00,10.626667,100.0,0.0,81.300000,0.0,0.0,11.123333,1.300000
2022-04-30 01:00:00+00:00,10.073333,100.0,0.0,88.633333,0.0,0.0,10.083333,1.300000
2022-04-30 02:00:00+00:00,9.803333,100.0,0.0,96.700000,0.0,0.0,9.286667,1.303333
2022-04-30 03:00:00+00:00,9.453333,100.0,0.0,100.000000,0.0,0.0,8.630000,1.300000
2022-04-30 04:00:00+00:00,9.725000,100.0,0.0,100.000000,0.0,0.0,8.835714,1.300000


## Data Nan -> Mean

In [11]:
method = "mean"
max_limit = {}
parameter = {}

data_refine_nan = BasicImputation(data_refine, method, max_limit, parameter).simpleMethod()

In [12]:
data_refine_nan.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2022-04-30 00:00:00+00:00 to 2022-04-30 23:00:00+00:00
Freq: H
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   out_blacksphere_temp  24 non-null     float64
 1   out_coai              24 non-null     float64
 2   out_h2s               24 non-null     float64
 3   out_humi              24 non-null     float64
 4   out_nh3               24 non-null     float64
 5   out_pm10              24 non-null     float64
 6   out_temp              24 non-null     float64
 7   out_ultraviolet_rays  24 non-null     float64
dtypes: float64(8)
memory usage: 1.7 KB


## Data Scale

In [13]:
# set parameter
scaler_root_dir ='./scaler/'

clean_train_data_param = 'NoClean'

data_name_x = "AdaRNN_data_x"
feature_list_x= list(data_refine_nan.columns)
x_scaler_file_path = './scaler/AdaRNN_data_x/NoClean/minmax/4457f465de3d0e17dd9f547db5b454b7/scaler.pkl'

split_ratio = 0.8
scaler_param='scale'
scale_method='minmax'

In [14]:
scaler_root_path_x = os.path.join(scaler_root_dir, data_name_x, clean_train_data_param)

inference_x, scaler_x = ml_scaler.get_scaled_test_data(data_refine_nan[feature_list_x], x_scaler_file_path, scaler_param)
scaler_y = ml_scaler.get_scaler_file(x_scaler_file_path)

#train_x, val_x, X_scaler_file_path = p3.get_train_val_data(data_refine_nan, feature_list_x, scaler_root_path_x, split_ratio, scaler_param, 
#                                                           scale_method, mode = 'Classification')

In [15]:
data_refine_nan

,out_blacksphere_temp,out_coai,out_h2s,out_humi,out_nh3,out_pm10,out_temp,out_ultraviolet_rays
time,,,,,,,,
2022-04-30 00:00:00+00:00,10.626667,100.000000,0.0,81.300000,0.0,0.000000,11.123333,1.300000
2022-04-30 01:00:00+00:00,10.073333,100.000000,0.0,88.633333,0.0,0.000000,10.083333,1.300000
2022-04-30 02:00:00+00:00,9.803333,100.000000,0.0,96.700000,0.0,0.000000,9.286667,1.303333
2022-04-30 03:00:00+00:00,9.453333,100.000000,0.0,100.000000,0.0,0.000000,8.630000,1.300000
2022-04-30 04:00:00+00:00,9.725000,100.000000,0.0,100.000000,0.0,0.000000,8.835714,1.300000
2022-04-30 05:00:00+00:00,9.936667,100.000000,0.0,100.000000,0.0,0.000000,9.116667,1.300000
2022-04-30 06:00:00+00:00,10.436667,100.000000,0.0,99.633333,0.0,0.000000,9.600000,1.303333
2022-04-30 07:00:00+00:00,10.956667,100.000000,0.0,86.600000,0.0,0.000000,10.723333,1.300000
2022-04-30 08:00:00+00:00,12.286667,99.833333,0.0,66.866667,0.0,1.400000,13.050000,1.303333


In [16]:
data = data_refine_nan.to_numpy()
data = data.reshape(1,24,8)
data.shape

(1, 24, 8)

# Data Loader

In [17]:
adarnn = ada_rnn.ClustAdaRnn(param)

{'x_features': 8, 'class_num': 1, 'dropout': 0.0, 'len_seq': 24, 'loss_type': 'adv', 'hidden_size': 64, 'num_layers': 2}


In [18]:
inference_loader = adarnn.create_inferenceloader(data, shuffle)

# Inference

In [19]:
## model load
adarnn.load_model(model_file_path)

In [20]:
## inference loader
pred_list = adarnn.inference(inference_loader)

[2023-03-16 16:27:37] - inference...
[2023-03-16 16:27:37] - Finished.


In [21]:
pred_list

array(-0.06552421, dtype=float32)

In [23]:
if scaler_param =='scale':
    base_df_for_inverse = pd.DataFrame(columns=[target_col], index=range(pred_list.size))
    base_df_for_inverse[target_col] = pred_list
    prediction_result = pd.DataFrame(scaler_y.inverse_transform(base_df_for_inverse), columns=target_col, index=base_df_for_inverse.index)
else:
    prediction_result = pd.DataFrame(data={"value": preds}, index=range(len(preds)))

ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,8)

In [24]:
base_df_for_inverse

,out_pm10
0,-0.065524
